In [ ]:
import datetime
from pathlib import Path

import yfinance as yf
import polars as pl
from tqdm import tqdm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import stock
from stock.kabutan import read_data_csv, read_financial_csv, get_code_list

In [ ]:
# watchlistの新しい定義を考える
# 週足で判定してみる？

def calc_for_watch_list(
    code: str,
    start_date: datetime.date | None = None,
    end_date: datetime.date = datetime.date.today(),
):
    df = read_data_csv(code, start_date=start_date, end_date=end_date, weekly=True)
    # 過去10週の値動きの大きさを計算
    window_size = 10
    avg_key = "avg{}".format(window_size)
    stddev_key = "stddev{}".format(window_size)
    df = df.with_columns(
        pl.col("close").rolling_mean(window_size=window_size).alias(avg_key),
        pl.col("close").rolling_std(window_size=window_size).alias(stddev_key),
    )

    # ギャップアップしている
    df = df.with_columns(
        (pl.col("close") > pl.col(avg_key) + pl.col(stddev_key)).alias("breakpoint")
    )

    # 直近の安値が安すぎない & 値幅が狭すぎない
    window_size = 10
    df = df.with_columns(
        (pl.col("close").rolling_min(window_size=window_size)).alias("min_close")
    ).with_columns(
        (
            (pl.col("min_close") > pl.col("close") * 0.7)
            & (pl.col("min_close") < pl.col("close") * 0.95)
        ).alias("price_range")
    )

    # 高値が多すぎない
    df = df.with_columns(
        pl.col("close")
        .rolling_map(
            function=lambda d: sum(d > d[-1]),
            window_size=30,
        )
        .alias("high_count")
    )

    # 出来高が増加（急増）
    window_size = 10
    df = df.with_columns(
        pl.col("volume").rolling_max(window_size=window_size).shift().alias("max_volume")
    )
    df = df.with_columns(
        (
            (pl.col("volume") > pl.col("max_volume") * 2)
            & (pl.col("volume") * pl.col("close") > 20000 * 100)
            & (pl.col("volume").rolling_max(window_size=30).shift() * 0.9 < pl.col("volume"))
        ).alias("volume_increase")
    )

    # watch listの条件判定
    df = df.with_columns(
        (
            pl.col("breakpoint")
            & pl.col("price_range")
            & pl.col("volume_increase")
            & (pl.col("high_count") < 7)
            & ((pl.col("close") >= pl.col("open")) | (pl.col("volume") > pl.col("max_volume") * 20))
        ).alias("watch_list")
    )

    # 直前にwatch list候補になっている場合はwatch listから除く
    df = df.with_columns(
        (
            (pl.col("watch_list").cast(int).rolling_max(window_size=5).shift() == 0)
            & pl.col("watch_list")
        ).alias("watch_list")
    )
    # 決算発表前後の日はwatch_listから除く
    return df


In [ ]:
code_list = get_code_list()

stacked_df = []
for code in tqdm(code_list):
    df = calc_for_watch_list(code)
    stacked_df.append(df.filter(pl.col("watch_list")).with_columns(pl.lit(code).alias("code")))

stacked_df = pl.concat(stacked_df)

In [ ]:
len(stacked_df)

In [ ]:
codes = sorted(stacked_df["code"].unique().to_list())
stacked_df

In [ ]:
idx = 50
print("code = {}, date = {}".format(stacked_df["code"][idx], stacked_df["date"][idx]))
stock.visualize.plot_chart_from_code(
    stacked_df["code"][idx], 
    start_date=stacked_df["date"][idx] - datetime.timedelta(days=365), 
    end_date=stacked_df["date"][idx] + datetime.timedelta(days=365), 
    weekly=True,
    before_days=365
).show()

In [ ]:
idx = 7
code = codes[idx]
fdf = stock.kabutan.read_financial_csv(code)
qfdf = fdf.filter(pl.col("duration") == 3).sort(pl.col("annoounce_date")).with_columns(
    (pl.col("eps") / pl.col("eps").shift(4) - 1.0).alias("growing_rate")
).filter(pl.col("growing_rate").is_not_null())

In [ ]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.0, row_heights=[0.7, 0.3], 
    specs=[[{"secondary_y": True}], [{"secondary_y": False}]]
)
fig.add_trace(
    go.Scatter(x=qfdf["annoounce_date"], y=qfdf["growing_rate"], mode="lines"), 
    row=1, col=1, secondary_y=True
)
df = stock.kabutan.read_data_csv(
    code, 
    start_date=qfdf["annoounce_date"][0] - datetime.timedelta(days=365),
    end_date=qfdf["annoounce_date"][-1] + datetime.timedelta(days=365),
    weekly=True,
)
fig = stock.visualize.plot_chart(df, fig=fig)
fig.show()

In [ ]:
ticker = yf.Ticker("AAPL")
res = ticker.history(interval="1d", period="max").reset_index()

In [ ]:
len(res)

In [ ]:
df = pl.DataFrame(
    {
        "date": res["Date"].to_list(),
        "open": res["Open"],
        "high": res["High"],
        "low": res["Low"],
        "close": res["Close"],
        "volume": res["Volume"],
        "dividends": res["Dividends"],
        "stock_splits": res["Stock Splits"],
    }
).with_columns(
    pl.col("date").cast(pl.Date)
)